In [1]:
from models import LinearRegression, BoostingModel
from train import ClimateDataset, loss_rmse
import pandas as pd
import torch
import numpy as np
import itertools
from tqdm.notebook import tqdm

def feature_importance(booster):
    return pd.DataFrame({
        'feature': booster.feature_name(),
        'importance': booster.feature_importance(importance_type='gain')
    }).sort_values('importance', ascending=False).reset_index(drop=True)

In [2]:
ds = ClimateDataset('aice', lead_times=[3], periods=[3])
model = BoostingModel(variables=ds.variables.copy())
model.fit(ds)
feature_importance(model.model)

,feature,importance
0,aice,243846.577600
1,cos_lat,16729.302294
2,ps,8547.605698
3,t850,7712.167528
4,sin_lon,7097.492949
5,olr,6993.244529
6,h500,6824.411523
7,hice,6786.388554
8,v850,6028.637073
9,sst,5990.564500


In [3]:
ds = ClimateDataset('swe', lead_times=[3], periods=list(range(44, 49)))
model = BoostingModel(variables=ds.variables.copy())
model.fit(ds)
feature_importance(model.model)

,feature,importance
0,t2min,540728356.0
1,swe,345903520.0
2,cos_lat,290962796.0
3,cos_lon,237870206.0
4,sin_lon,221507471.0
5,tvl,218294927.0
6,sdor,189033583.0
7,h500,168372332.0
8,t2,158488576.0
9,ww,147749781.0


In [2]:
def score_model(model, X, y, lat, climate, s, mx=1):
    model.__fit__(X[:, s], y)
    y_pred = torch.clamp(model.predict(X[:, s]) + climate, 0, max=mx)
    return loss_rmse(y_pred, y + climate, lat)

def shapley_approximation(model, variables, X, y, lat, climate, mx=1, n_samples=500):
    n = X.shape[1]
    shapley_values = np.zeros(n)
    
    for _ in tqdm(range(n_samples)):
        order = np.random.permutation(n)
        cur_set = []
        prev_score = score_model(model, X, y, lat, climate, cur_set, mx=mx)
        for i, idx in enumerate(order):
            cur_set.append(idx)
            cur_score = score_model(model, X, y, lat, climate, cur_set, mx=mx)
            for idx in cur_set:
                shapley_values[idx] += (cur_score - prev_score) / (i + 1)
            prev_score = cur_score
    shapley_values /= n_samples / 1000

    return pd.DataFrame({
        'variable': variables,
        'SHAP': -shapley_values
    }).sort_values('SHAP', ascending=False).reset_index(drop=True)

In [3]:
ds = ClimateDataset('aice', lead_times=[3], periods=[3], normed=True, use_cache=True)
ds.set_variables([v for v in ds.variables if v not in ['cos_period', 'sin_period']])
X, y, lat, climate = ds.load_all()
shapley_approximation(LinearRegression([]), ds.variables.copy(), X, y, lat, climate, n_samples=1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

,variable,SHAP
0,aice,5.786736
1,cos_lat,3.000968
2,ps,1.675598
3,sst,1.665752
4,t2min,1.640753
5,olr,1.636141
6,t850,1.621056
7,uv10,1.587313
8,t2max,1.576886
9,prec,1.547284
